In [ ]:
import os

import contextily as ctx
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shapely
import xarray as xr

from matplotlib import colors
from shapely.affinity import translate
from shapely.geometry import LineString, Point, Polygon
from shapely.ops import substring

### Functions

In [ ]:
def bitwiseMask(ds):
    # Fow now, eliminate the really bad stuff
    mask = np.where((ds.classification > 2) & (ds.geolocation_qual < 2**16) &
                    (np.abs(ds.cross_track) > 10000) & (np.abs(ds.cross_track) < 60000))[0]
    
    print(mask.shape)
    return mask

In [ ]:
def makeGDF(ds, mask, data_vars):
    '''
    This function takes the pixel cloud xarray object, makes a masked
    GeoDataFrame, renames columns as needed, set the CRS, reprojects
    the CRS, and returns a GeoDataFrame.
    '''
    # Subset xarray, convert to masked DataFrame
    xarr = ds[data_vars]
    df = xarr.to_dataframe().loc[mask].reset_index()

    # Create GDF
    gdf_PIXC = gpd.GeoDataFrame(df,
                                geometry=gpd.points_from_xy(df.longitude,
                                                            df.latitude),
                                crs="EPSG:4326") # PIXC has no native CRS, setting same as River_SP

    if 'classification' in gdf_PIXC.columns:
        gdf_PIXC.rename(columns={'classification': 'klass'}, inplace=True)
    
    # Convert the crs to WGS 84 / UTM zone 18N
    gdf_PIXC = gdf_PIXC.to_crs(epsg='32618')
    
    return gdf_PIXC

In [ ]:
def makePseudoPixels(pixel, segment_ln, height):
    
    # Get pixel geometry
    pixel_pt = pixel.geometry #[pixel.index[0]]
    
    # Calculate width
    width = pixel.pixel_area/height #[pixel.index[0]]/height
    
#     # Get distance along nadir track closest to pixel
#     dist = segment_ln.project(pixel_pt)
    
#     # Get coordinate of point along nadir closest to pixel
#     projection = substring(segment_ln, dist, 0).coords[1]
    
#     # Create linestring from pixel to closest point on nadir track
#     orthogonal = LineString([pixel_pt.coords[0], projection])
    orthogonal = shapely.shortest_line(pixel_pt, segment_ln)
    
    # return orthogonal
    
    # Make line parallel to orthogonal at correct height
    up = orthogonal.parallel_offset(distance=height/2, side='right')
    down = orthogonal.parallel_offset(distance=height/2, side='left')
    
    # Get coords for inner edge
    one_coord = substring(up, start_dist=0, end_dist=width/2).coords[1]
    two_coord = substring(down, start_dist=0, end_dist=width/2).coords[1]
    
    # Get inner and outer edges of polygon
    inner_edge = LineString([two_coord, one_coord])
    outer_edge = inner_edge.parallel_offset(distance=width, side='right')
    
    # Get coords for outer edge
    three_coord = outer_edge.coords[0]
    four_coord = outer_edge.coords[1]
    
    # Make pseudo pixel
    pseudo_pixel = Polygon((one_coord, two_coord, three_coord, four_coord, one_coord))
    
    return pseudo_pixel

In [ ]:
def readNHD(index):
    ## Set-up
    mdata_path = '/nas/cee-water/cjgleason/fiona/narrow_rivers_PIXC/data/'
# NEED WITH WATERBODY FOR CUSHMAN!!!!
    prep_path = '/nas/cee-water/cjgleason/fiona/narrow_rivers_PIXC_data/NHD_prepped_with_waterbody/' # _with_waterbody

    # Define dtypes for lookup tables to preserve leading zeros
    dtype_dic= {'HUC4': str, 'HUC2': str, 'toBasin': str, 'level': str}
    # Read in HUC lookup table
    lookup = pd.read_csv(os.path.join(mdata_path, 'HUC4_lookup_no_great_lakes.csv'), dtype=dtype_dic)

    # Get current HUC2 and HUC4 IDs
    hu2 = 'HUC2_' + lookup.loc[index,'HUC4'][0:2]
    hu4 = 'NHDPLUS_H_' + lookup.loc[index,'HUC4'] + '_HU4_GDB'
    
    # Set data filepath
    file_path = os.path.join(prep_path, hu2, hu4 + '_prepped_with_waterbody.gpkg') # _with_waterbody

    ## Read in prepped NHD flowlines
    features = ['NHDPlusID', 'GNIS_Name', 'LengthKM', 'WidthM', 'Bin', 'geometry']
    basin = gpd.read_file(filename=file_path, columns=features, engine='pyogrio')
    
    # Convert CRS to WGS 84 / UTM zone 18N
    basin = basin.to_crs(epsg='32618')
    
    # Drop reaches that are shorter than their width
    basin = basin[basin['LengthKM']*1000 > basin['WidthM']]
    
    # Make geometry 2D LineStrings
    basin['geometry'] = basin.geometry.explode().force_2d()
    return basin

### Pixel Cloud

In [ ]:
test = '../data/small_rivers/mar_2024_ver_c/SWOT_L2_HR_PIXC_012_298_080L_20240317T084237_20240317T084248_PIC0_01.nc'

In [ ]:
# Read in xarray
ds_PIXC = xr.open_mfdataset(paths=test, group = 'pixel_cloud', engine='h5netcdf')

In [ ]:
ds_PIXC

In [ ]:
# Read in xarray
ds_GLOB = xr.open_mfdataset(paths=test, engine='h5netcdf')

In [ ]:
# ds_GLOB

In [ ]:
pass_num = ds_GLOB.pass_number
pass_num

In [ ]:
swath_side = ds_GLOB.swath_side
swath_side

In [ ]:
# Make dict for legend labels
flags = ds_PIXC.classification.flag_meanings.split() # extract each flag meaning
codes = {idx:k for idx, k in enumerate(flags, start=1)}

In [ ]:
# Make mask
mask = bitwiseMask(ds_PIXC)

In [ ]:
variables = ['azimuth_index', 'range_index', 'cross_track',
             'pixel_area', 'classification']

In [ ]:
# If dataframe not empty after filtering
if mask.shape != (0,):
    # Make PIXC
    gdf_PIXC = makeGDF(ds=ds_PIXC, mask=mask, data_vars=variables)
    # # Append to list
    # d.append(gdf_PIXC)

In [ ]:
gdf_PIXC

In [ ]:
# Get single pixel for selecting correct nadir segment
pixel = gdf_PIXC.sample(n=1, random_state=0)

In [ ]:
# Get geometry of signle pixel
pixel_pt = pixel.geometry[pixel.index[0]]

### Nadir track

In [ ]:
# Read in nadir (science orbit)
nadir = gpd.read_file('/nas/cee-water/cjgleason/data/SWOT/swath/swot_science_hr_Aug2021-v05_shapefile_nadir/swot_science_hr_2.0s_4.0s_Aug2021-v5_nadir.shp')

In [ ]:
# Convert CRS to WGS 84 / UTM zone 18N
nadir = nadir.to_crs(epsg=32618)

In [ ]:
# Find candidate nadir segments
candidates = nadir[nadir['ID_PASS'] == pass_num]

In [ ]:
# Find distance from each candidate to single pixel
candidates['dist'] = candidates.loc[:,'geometry'].distance(pixel_pt)

In [ ]:
# Get nadir segment closest to single pixel
nadir_segment = candidates[candidates.dist == candidates.dist.min()]

In [ ]:
# Get nadir segment geoemtry
nadir_segment_ln = nadir_segment.geometry[nadir_segment.index[0]]

### Make pseudo pixels

In [ ]:
# Set pixel height
height = 22

In [ ]:
t = gdf_PIXC.loc[0]

In [ ]:
t.pixel_area

In [ ]:
test = gdf_PIXC.loc[100]

In [ ]:
test

In [ ]:
ortho = makePseudoPixels(test, nadir_segment_ln, height)

In [ ]:
ortho

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
gpd.GeoSeries(nadir_segment_ln).plot(ax=ax)
gpd.GeoSeries(test.geometry).plot(ax=ax, color='k')
# gpd.GeoSeries(test.geometry).plot(ax=ax)
# gpd.GeoSeries(test_poly).plot(ax=ax, alpha=0.5)
gpd.GeoSeries(ortho).plot(ax=ax, alpha=0.5)

In [ ]:
subset = gdf_PIXC.iloc[0:50000,:]

In [ ]:
subset

In [ ]:
# Make pseudo pixels
subset['pseudo_pixel'] = subset.apply(func=makePseudoPixels,
                                          args=(nadir_segment_ln,
                                                height), axis=1)
# test = gdf_PIXC.loc[0].apply(func=makePseudoPixels,
#                                           args=(nadir_segment_ln,
#                                                 height))

In [ ]:
subset

In [ ]:
# Make pseudo pixels
gdf_PIXC['pseudo_pixel'] = gdf_PIXC.apply(func=makePseudoPixels,
                                          args=(nadir_segment_ln,
                                                height), axis=1)

In [ ]:
gdf_PIXC

In [ ]:
pseudo = gdf_PIXC.drop(columns='geometry').set_geometry('pseudo_pixel')

In [ ]:
bound_box = pseudo.total_bounds

In [ ]:
bound_box

In [ ]:
pseudo

In [ ]:
# Set slurm job index
# (see: nas/cee-water/cjgleason/fiona/narrow_rivers_PIXC/data/HUC4_lookup_no_great_lakes.csv)
i = 4 # HUC4 0108, Connecticut

In [ ]:
basin = readNHD(index=i)

In [ ]:
basin

In [ ]:
buffer = basin.copy()

In [ ]:
buffer

In [ ]:
# Buffer reach by 1/2 channel width
buffer['geometry'] = buffer['geometry'].buffer(distance=(buffer['WidthM']/2), cap_style='flat')

### Plots

In [ ]:
savepath = '/nas/cee-water/cjgleason/fiona/narrow_rivers_PIXC/pseudo_pixels/'

In [ ]:
# Make color palette
# palette = {2: 'darkolivegreen', 3: 'slateblue', 4: 'steelblue', 5: 'hotpink',
#            6: 'saddlebrown', 7: 'darkslategray'}
palette = {2: 'darkkhaki', 3: 'cornflowerblue', 4: 'blue', 5: 'hotpink',
           6: 'lightsalmon', 7: 'lightblue'}

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))

pseudo.plot(ax=ax, alpha=0.3, color='y')
gpd.GeoSeries(nadir_segment_ln).plot(ax=ax, color='k')

basin.plot(ax=ax, alpha=0.5)
subset.plot(ax=ax, markersize=1, color='k')

In [ ]:
pseudo_c = pseudo
buffer_c = buffer.clip(mask=bound_box)

nums = list(np.unique(pseudo.klass).astype('int'))
labels = [codes[x] for x in nums]

# Make cmap
cmap = colors.ListedColormap([palette[x] for x in nums])

fig, ax = plt.subplots(figsize=(15,15))
# basin.clip(mask=bound_box).plot(ax=ax, alpha=0.5)
buffer_c.plot(ax=ax, alpha=0.5, color='w')
pseudo_c.plot(ax=ax,
            column=pseudo_c.klass,
            categorical=True,
            cmap=cmap,
            legend=True,
            legend_kwds={'labels': labels,
                         'framealpha': 1, 
                         'title_fontsize': 'medium',
                         'loc': 'upper left'})

# Basemap
ctx.add_basemap(ax, crs=gdf_PIXC.crs, source=ctx.providers.CartoDB.DarkMatter, alpha=0.9)

# gpd.GeoSeries(nadir_segment_ln).plot(ax=ax, color='k')
# subset.plot(ax=ax, markersize=1, color='k')

In [ ]:
# bounds = [690000, 4695000, 700000, 4705000]

# pseudo_c = pseudo.clip(mask=bounds)
# buffer_c = buffer.clip(mask=bounds)

# nums = list(np.unique(pseudo.klass).astype('int'))
# labels = [codes[x] for x in nums]

# # Make cmap
# cmap = colors.ListedColormap([palette[x] for x in nums])

# fig, ax = plt.subplots(figsize=(15,15))
# pseudo.plot(ax=ax, alpha=0.9,
#             column=pseudo.klass,
#             categorical=True,
#             cmap=cmap,
#             legend=True,
#             legend_kwds={'labels': labels,
#                          'framealpha': 1, 
#                          'title_fontsize': 'medium',
#                          'loc': 'lower left'})
# # basin.clip(mask=bound_box).plot(ax=ax, alpha=0.5)
# # buffer.clip(mask=bound_box).plot(ax=ax, alpha=0.2, color='w')

# # Basemap
# # ctx.add_basemap(ax, crs=gdf_PIXC.crs, source=ctx.providers.CartoDB.DarkMatter, alpha=0.9)

# # gpd.GeoSeries(nadir_segment_ln).plot(ax=ax, color='k')
# # subset.plot(ax=ax, markersize=1, color='k')

In [ ]:
# bounds = [690000, 4695000, 700000, 4705000]
bounds = [692000, 4694000, 702000, 4704000]

pseudo_c = pseudo.clip(mask=bounds)
buffer_c = buffer.clip(mask=bounds)

nums = list(np.unique(pseudo.klass).astype('int'))
labels = [codes[x] for x in nums]

# Make cmap
cmap = colors.ListedColormap([palette[x] for x in nums])

fig, ax = plt.subplots(figsize=(15,15))
# basin.clip(mask=bound_box).plot(ax=ax, alpha=0.5)
buffer_c.plot(ax=ax, alpha=0.5, color='w')
pseudo_c.plot(ax=ax,
            column=pseudo_c.klass,
            categorical=True,
            cmap=cmap,
            legend=True,
            legend_kwds={'labels': labels,
                         'framealpha': 1, 
                         'title_fontsize': 'medium',
                         'loc': 'upper left'})

# Basemap
ctx.add_basemap(ax, crs=gdf_PIXC.crs, source=ctx.providers.CartoDB.DarkMatter, alpha=0.9)

plt.xlim(692000, 702000)
plt.ylim(4694000, 4704000)

plt.savefig(fname=savepath + 'conn_great_pond.png', bbox_inches='tight')

In [ ]:
bounds = [692000, 4684000, 702000, 4694000]

pseudo_c = pseudo.clip(mask=bounds)
buffer_c = buffer.clip(mask=bounds)

nums = list(np.unique(pseudo.klass).astype('int'))
labels = [codes[x] for x in nums]

# Make cmap
cmap = colors.ListedColormap([palette[x] for x in nums])

fig, ax = plt.subplots(figsize=(15,15))
# basin.clip(mask=bound_box).plot(ax=ax, alpha=0.5)
buffer_c.plot(ax=ax, alpha=0.8, color='w')
pseudo_c.plot(ax=ax,
            column=pseudo_c.klass,
            categorical=True,
            cmap=cmap,
            legend=True,
            legend_kwds={'labels': labels,
                         'framealpha': 1, 
                         'title_fontsize': 'medium',
                         'loc': 'upper left'})

# Basemap
ctx.add_basemap(ax, crs=gdf_PIXC.crs, source=ctx.providers.CartoDB.DarkMatter, alpha=0.9)

plt.xlim(692000, 702000)
plt.ylim(4684000, 4694000)

plt.savefig(fname=savepath + 'conn_hadley.png', bbox_inches='tight')

In [ ]:
bounds = [698000, 4688000, 700000, 4690000]

pseudo_c = pseudo.clip(mask=bounds)
buffer_c = buffer.clip(mask=bounds)

nums = list(np.unique(pseudo.klass).astype('int'))
labels = [codes[x] for x in nums]

# Make cmap
cmap = colors.ListedColormap([palette[x] for x in nums])

fig, ax = plt.subplots(figsize=(15,15))
# basin.clip(mask=bound_box).plot(ax=ax, alpha=0.5)
buffer_c.plot(ax=ax, alpha=0.8, color='w')
pseudo_c.plot(ax=ax, alpha=0.9,
            column=pseudo_c.klass,
            categorical=True,
            cmap=cmap,
            legend=True,
            legend_kwds={'labels': labels,
                         'framealpha': 1, 
                         'title_fontsize': 'medium',
                         'loc': 'upper left'})

# Basemap
ctx.add_basemap(ax, crs=gdf_PIXC.crs, source=ctx.providers.CartoDB.DarkMatter, alpha=0.9)

plt.xlim(698000, 700000)
plt.ylim(4688000, 4690000)

plt.savefig(fname=savepath + 'conn_fort.png', bbox_inches='tight')

In [ ]:
nums = list(np.unique(pseudo.klass).astype('int'))
labels = [codes[x] for x in nums]

# Make cmap
cmap = colors.ListedColormap([palette[x] for x in nums])

fig, ax = plt.subplots(figsize=(8,8))
pseudo.plot(ax=ax, alpha=0.8,
            column=pseudo.klass,
            categorical=True,
            cmap=cmap,
            legend=True,
            legend_kwds={'labels': labels,
                         'framealpha': 1, 
                         'title_fontsize': 'medium',
                         'loc': 'lower left'})
basin.plot(ax=ax, alpha=0.5)
buffer.plot(ax=ax, alpha=0.3)
# gpd.GeoSeries(nadir_segment_ln).plot(ax=ax, color='k')
subset.plot(ax=ax, markersize=1, color='k')

plt.xlim(683961, 684677)
plt.ylim(4728171, 4728924)

In [ ]:
nums = list(np.unique(pseudo.klass).astype('int'))
labels = [codes[x] for x in nums]

# Make cmap
cmap = colors.ListedColormap([palette[x] for x in nums])

fig, ax = plt.subplots(figsize=(8,8))

pseudo.plot(ax=ax, alpha=0.8,
            column=pseudo.klass,
            categorical=True,
            cmap=cmap,
            legend=True,
            legend_kwds={'labels': labels,
                         'framealpha': 1, 
                         'title_fontsize': 'medium',
                         'loc': 'lower left'})
basin.plot(ax=ax, alpha=0.5)
buffer.plot(ax=ax, alpha=0.3)
subset.plot(ax=ax, markersize=1, color='k')

plt.xlim(688337, 689053)
plt.ylim(4729756, 4730510)